In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import matplotlib
import matplotlib.pyplot as plt
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from fft3d import FFT3Dfield_new
from EnergySpectrum import EnergySpectrum

In [3]:
%%px

Nx = isotropic1024coarse['nx']
Ny = isotropic1024coarse['ny']
Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']
Ly = isotropic1024coarse['ly']
Lz = isotropic1024coarse['lz']

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))
    
# Computational Domain

nx=Nx//nproc
ny=Ny
nz=Nz
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:4] 
n_proc = 8
rank = 0


In [4]:
%%px

folder = "/home/idies/workspace/scratch"
filename = "ref-strainrate-"+str(rank)+".npz"
file = folder + "/" + filename

comm.Barrier(); t1=MPI.Wtime()
content = np.load(file)
    
S2 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
    
if(int(content['nproc'])!=nproc):
    print("Unmatched number of processes. Must first pre-process to adequate number of process")
S2[:,:,:] = content['S2']
    
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Finished loading")
    sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

[stdout:4] 
Finished loading
Load from disk: 18.26 seconds


In [5]:
%%px

folder = "/home/idies/workspace/scratch"
filename = "ref-enstrophy-"+str(rank)+".npz"
file = folder + "/" + filename

comm.Barrier(); t1=MPI.Wtime()
content = np.load(file)
    
w2 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
    
if(int(content['nproc'])!=nproc):
    print("Unmatched number of processes. Must first pre-process to adequate number of process")
w2[:,:,:] = content['w2']
    
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    print("Finished loading")
    sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))

[stdout:4] 
Finished loading
Load from disk: 22.49 seconds


In [10]:
%%px

fw2 = w2.flatten()

In [11]:
%%px

fS2 = S2.flatten()

In [12]:
%%px

jw2s2 = np.dstack((fw2,fS2))

In [14]:
%%px 

print(jw2s2[0].shape)

[stdout:0] (134217728, 2)
[stdout:1] (134217728, 2)
[stdout:2] (134217728, 2)
[stdout:3] (134217728, 2)
[stdout:4] (134217728, 2)
[stdout:5] (134217728, 2)
[stdout:6] (134217728, 2)
[stdout:7] (134217728, 2)


In [17]:
%%px

avgEpsilon = np.average(S2)
avgEpsilonGl=np.zeros(1,dtype='float32')

comm.Reduce([avgEpsilon,MPI.REAL],[avgEpsilonGl,MPI.REAL],op=MPI.SUM)
avgEpsilon = avgEpsilonGl[0]/nproc
avgEpsilon = comm.bcast(avgEpsilon, root=0)

##########################

minS2 = S2.min()
maxS2 = S2.max()

minS2Gl=np.zeros(nproc,dtype='float32')
maxS2Gl=np.zeros(nproc,dtype='float32')

comm.Allgather([minS2,MPI.REAL],[minS2Gl,MPI.REAL])
comm.Allgather([maxS2,MPI.REAL],[maxS2Gl,MPI.REAL])

minS2 = minS2Gl.min()
maxS2 = maxS2Gl.max()

comm.Barrier()

if rank==0:
    print("<S_ij S_ij> : "+str(avgEpsilon))
    print("min epsilon/<epsilon> : "+str(minS2/avgEpsilon))
    print("min epsilon/<epsilon> : "+str(maxS2/avgEpsilon))

[stdout:4] 
<S_ij S_ij> : 249.217727661
min epsilon/<epsilon> : 3.98489813944e-05
min epsilon/<epsilon> : 225.010358768


In [19]:
%%px

avgOmega = np.average(w2)
avgOmegaGl=np.zeros(1,dtype='float32')

comm.Reduce([avgOmega,MPI.REAL],[avgOmegaGl,MPI.REAL],op=MPI.SUM)
avgOmega = avgOmegaGl[0]/nproc
avgOmega = comm.bcast(avgOmega, root=0)

##########################

minw2 = w2.min()
maxw2 = w2.max()

minw2Gl=np.zeros(nproc,dtype='float32')
maxw2Gl=np.zeros(nproc,dtype='float32')

comm.Allgather([minw2,MPI.REAL],[minw2Gl,MPI.REAL])
comm.Allgather([maxw2,MPI.REAL],[maxw2Gl,MPI.REAL])

minw2 = minw2Gl.min()
maxw2 = maxw2Gl.max()

comm.Barrier()

if rank==0:
    print("<0.5*omega^2> : "+str(avgOmega/2))
    print("min w2/<w^2> : "+str(minw2/avgOmega))
    print("min w2/<w^2> : "+str(maxw2/avgOmega))

[stdout:4] 
<0.5*omega^2> : 249.21774292
min w2/<w^2> : 1.43020472082e-07
min w2/<w^2> : 675.170126447


In [20]:
%%px

#numpy.linspace(start, stop, num=50, endpoint=True, retstep=False, dtype=None)

nE = 100
nO = 100

linE = np.linspace(minS2/avgEpsilon,maxS2/avgEpsilon, nE)
linO = np.linspace(minw2/avgOmega,maxw2/avgOmega, nO)

In [ ]:
%%px

